In [2]:
!pip install pathway bokeh --quiet

In [6]:
!pip install panel

   ---------------------------------------- 0.0/29.6 MB ? eta -:--:--
   ---------------------------------------- 0.3/29.6 MB ? eta -:--:--
    --------------------------------------- 0.5/29.6 MB 858.8 kB/s eta 0:00:34
   - -------------------------------------- 1.0/29.6 MB 1.7 MB/s eta 0:00:18
   -- ------------------------------------- 1.6/29.6 MB 2.0 MB/s eta 0:00:14
   -- ------------------------------------- 1.6/29.6 MB 2.0 MB/s eta 0:00:14
   -- ------------------------------------- 1.6/29.6 MB 2.0 MB/s eta 0:00:14
   -- ------------------------------------- 1.8/29.6 MB 1.3 MB/s eta 0:00:22
   -- ------------------------------------- 2.1/29.6 MB 1.3 MB/s eta 0:00:22
   --- ------------------------------------ 2.4/29.6 MB 1.2 MB/s eta 0:00:22
   --- ------------------------------------ 2.6/29.6 MB 1.3 MB/s eta 0:00:22
   --- ------------------------------------ 2.9/29.6 MB 1.2 MB/s eta 0:00:22
   --- ------------------------------------ 2.9/29.6 MB 1.2 MB/s eta 0:00:22
   ---- ---

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [45]:
df = pd.read_csv('dataset.csv')

In [46]:
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [31]:
# 📦 Step 1: Imports
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource

output_notebook()


Loading BokehJS ...

In [50]:
import json, textwrap, nbformat as nbf
df = pd.read_csv('dataset.csv')

# Construct notebook
nb = nbf.v4.new_notebook()

cells = []

cells.append(nbf.v4.new_code_cell(textwrap.dedent("""\
# 📦 Step 1: Imports
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource

output_notebook()
""")))

cells.append(nbf.v4.new_code_cell(textwrap.dedent("""\
# 📂 Step 2: Load Dataset
try:
    df = pd.read_csv("dataset.csv")  # Ensure this file is in the same folder
except FileNotFoundError:
    print("Please make sure 'dataset.csv' is in the current directory.")
    df = pd.DataFrame()  # Create empty DataFrame to avoid errors
""")))

cells.append(nbf.v4.new_code_cell(textwrap.dedent("""\
# 🧹 Step 3: Preprocessing
if not df.empty:
    df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'])
    df['OccupancyRate'] = df['Occupancy'] / df['Capacity']
    df['VehicleTypeWeight'] = df['VehicleType'].map({'car': 1.0, 'bike': 0.5, 'truck': 1.5})
    df['TrafficLevel'] = df['TrafficConditionNearby'].map({'low': 1, 'medium': 2, 'high': 3})
    df = df.sort_values(by=['SystemCodeNumber', 'Timestamp'])
""")))

cells.append(nbf.v4.new_code_cell(textwrap.dedent("""\
# 📈 Step 4: Model 1 – Baseline Linear Pricing
def baseline_model(df, alpha=5):
    df['BaselinePrice'] = 10 + alpha * df['OccupancyRate']
    return df

if not df.empty:
    df = baseline_model(df)
""")))

cells.append(nbf.v4.new_code_cell(textwrap.dedent("""\
# ⚙️ Step 5: Model 2 – Demand‑Based Pricing
def demand_based_model(df, alpha=1.2, beta=0.8, gamma=0.6, delta=2.0, epsilon=1.5, lam=0.5):
    demand = (
        alpha * df['OccupancyRate'] +
        beta * df['QueueLength'] -
        gamma * df['TrafficLevel'] +
        delta * df['IsSpecialDay'] +
        epsilon * df['VehicleTypeWeight']
    )
    demand = (demand - demand.min()) / (demand.max() - demand.min())  # Normalize
    df['Demand'] = demand
    df['DynamicPrice'] = 10 * (1 + lam * demand)
    df['DynamicPrice'] = df['DynamicPrice'].clip(5, 20)
    return df

if not df.empty:
    df = demand_based_model(df)
""")))

cells.append(nbf.v4.new_code_cell(textwrap.dedent("""\
# 📊 Step 6: Simple Bokeh Plot
if not df.empty:
    lot = df['SystemCodeNumber'].iloc[0]
    lot_df = df[df['SystemCodeNumber'] == lot]

    source = ColumnDataSource(lot_df)

    p = figure(x_axis_type='datetime', title=f"Prices Over Time for Lot: {lot}", width=800, height=300)
    p.line(x='Timestamp', y='BaselinePrice', source=source, line_width=2, color='green', legend_label="Baseline Price")
    p.line(x='Timestamp', y='DynamicPrice', source=source, line_width=2, color='blue', legend_label="Dynamic Price")
    p.legend.location = "top_left"

    show(p)
""")))

cells.append(nbf.v4.new_markdown_cell(textwrap.dedent("""\
## Summary

This notebook demonstrates a minimal dynamic pricing workflow:

1. **Baseline Pricing** – linear function of occupancy.  
2. **Demand‑Based Pricing** – accounts for queue length, traffic, special days, and vehicle type.  
3. **Bokeh visualisation** – compares baseline vs. dynamic price for the first parking lot.

Simply place `dataset.csv` alongside this notebook and run all cells. Prices are bounded between \$5 and \$20 for stability.
""")))

nb['cells'] = cells

# Save notebook to file
output_path = "dataset.ipynb"
with open(output_path, "w", encoding="utf-8") as f:
    nbf.write(nb, f)

output_path
import os, nbformat as nbf, textwrap

# ---- build the notebook cells exactly as before ----
nb = nbf.v4.new_notebook()
# [... add your cells to nb['cells'] …]

# 1️⃣  Use a raw string for the markdown that contains “\$”
markdown_src = textwrap.dedent(r"""
## Summary

Prices are bounded between \$5 and \$20 for stability.
""")
nb['cells'].append(nbf.v4.new_markdown_cell(markdown_src))

# 2️⃣  Decide where to save (here: current working directory)
output_path = "Simple_Dynamic_Pricing_Notebook.ipynb"

# 3️⃣  (Optional) ensure any parent directory exists
# os.makedirs(os.path.dirname(output_path), exist_ok=True)

# 4️⃣  Write the file
with open(output_path, "w", encoding="utf-8") as f:
    nbf.write(nb, f)

print(f"Notebook saved to → {os.path.abspath(output_path)}")



Notebook saved to → C:\Users\HP\Simple_Dynamic_Pricing_Notebook.ipynb


<>:93: SyntaxWarning: invalid escape sequence '\$'
<>:93: SyntaxWarning: invalid escape sequence '\$'
C:\Users\HP\AppData\Local\Temp\ipykernel_17160\1942152337.py:93: SyntaxWarning: invalid escape sequence '\$'
  Simply place `dataset.csv` alongside this notebook and run all cells. Prices are bounded between \$5 and \$20 for stability.
